In [26]:
#%pip install langchain_aws langchain langchain-community gremlin-python

In [43]:
%seed --model Property_Graph --dataset airport --run

Dropdown(description='Source type:', options=('', 'samples', 'custom'), style=DescriptionStyle(description_wid…

Dropdown(description='Data model:', layout=Layout(display='none', visibility='hidden'), options=('', 'property…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Language:', layout=Layout(display='none', visibility='hidden'), options=('', 'opencypher…

Dropdown(description='Data set:', layout=Layout(display='none', visibility='hidden'), options=(), style=Descri…

...

Dropdown(description='Location:', layout=Layout(display='none', visibility='hidden'), options=('Local', 'S3'),…

FileChooser(path='/home/ec2-user/SageMaker', filename='', title='', show_hidden=False, select_desc='Select', c…

Button(description='Submit', layout=Layout(visibility='hidden'), style=ButtonStyle())

Output()

Output()

In [50]:
%%gremlin
g.V().hasLabel('airport').limit(10).valueMap(true)

In [51]:
%%gremlin
g.E().hasLabel('route').limit(10).valueMap(true)

In [66]:
from gremlin_python.driver import client
from gremlin_python.driver.protocol import GremlinServerError

neptune_endpoint = "db-neptune-1.cluster-ro-choicos2alvz.us-east-1.neptune.amazonaws.com"
neptune_port = 8182

# Replace NeptuneGraph with a custom class
class CustomNeptuneGraph:
    def __init__(self, host, port, use_https=True):
        protocol = 'wss' if use_https else 'ws'
        self.endpoint = f"{protocol}://{host}:{port}/gremlin"
        self.client = client.Client(self.endpoint, 'g')

    def query(self, query_string):
        try:
            result = self.client.submit(query_string).all().result()
            return result
        except GremlinServerError as e:
            print(f"Error executing query: {e}")
            return None

# Set up the custom Neptune graph connection
graph = CustomNeptuneGraph(
    host=neptune_endpoint,
    port=neptune_port,
    use_https=True
)

# The rest of your code remains the same
prompt_template = """
Given the following question about an airport database:
{question}

Generate ONLY the Gremlin query to answer this question. The database has vertices labeled as 'airport' 
and edges labeled as 'route'. Do not include any explanations or comments, just the raw Gremlin query.

Gremlin Query:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["question"])

# Create an LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Function to ask questions using the chain and execute the query
def ask_question(question):
    # Generate the Gremlin query
    response = chain.run(question)
    
    # Extract only the Gremlin query (remove any explanations or comments)
    query = response.strip().split('\n')[-1]  # Get the last line of the response
    print(f"Generated Query: {query}")
    
    # Execute the query on the Neptune database
    result = graph.query(query)
    return result


In [67]:
# Example question
question = "How many outgoing routes does the Atlanta city have?"
response = ask_question(question)
print(f"Query Result: {response}")

Generated Query: g.V().has('airport', 'city', 'Atlanta').outE('route').count()
Query Result: [242]


In [60]:
# Example question
question = "Get a list of all countries with airports"
response = ask_question(question)
print(f"Query Result: {response}")

Generated Query: g.V().hasLabel('airport').out('route').values('country').dedup()
Query Result: ['US', 'TR', 'DE', 'MX', 'PR', 'CA', 'UK', 'FR', 'JP', 'AE', 'IE', 'CN', 'IT', 'NL', 'ES', 'CH', 'BE', 'PT', 'BS', 'BB', 'VI', 'BM', 'NG', 'JM', 'BQ', 'AW', 'CO', 'BZ', 'QA', 'DO', 'CR', 'HN', 'TC', 'PA', 'SV', 'LC', 'EC', 'NI', 'HT', 'KY', 'BR', 'KR', 'ZA', 'AR', 'PE', 'CL', 'SX', 'VE', 'GT', 'CU', 'GP', 'IS', 'RU', 'DK', 'HK', 'CV', 'NO', 'MQ', 'IL', 'AU', 'KN', 'TT', 'SE', 'GH', 'MA', 'IN', 'AT', 'SA', 'SN', 'ET', 'KW', 'TG', 'NZ', 'TW', 'AZ', 'UA', 'FI', 'AG', 'CW', 'VC', 'PH', 'HU', 'LV', 'GY', 'KE', 'PL', 'RS', 'EG', 'JO', 'GD', 'GU', 'SG', 'CK', 'FJ', 'PF', 'SR', 'PY', 'BO', 'UY', 'GR', 'CI', 'CZ', 'WS', 'AS', 'MH', 'KI', 'MY', 'PG', 'BH', 'VN', 'LU', 'KZ', 'UG', 'RO', 'LB', 'SL', 'LY', 'BD', 'UZ', 'IR', 'MU', 'SC', 'AL', 'LT', 'MD', 'ME', 'TJ', 'ZM', 'DJ', 'SD', 'TZ', 'MK', 'AF', 'CM', 'DZ', 'BF', 'NE', 'CY', 'TN', 'NP', 'OM', 'ID', 'BY', 'MV', 'RW', 'SI', 'GA', 'TD', 'CD', 'ML', 'MR

In [62]:
# Example question
question = "Count airports by country"
response = ask_question(question)
print(f"Query Result: {response}")

Generated Query: g.V().hasLabel('airport').groupCount().by('country')
Query Result: [{'PR': 7, 'PT': 19, 'PW': 1, 'PY': 2, 'QA': 1, 'AE': 10, 'AF': 5, 'AG': 1, 'AI': 1, 'AL': 1, 'AM': 2, 'AO': 14, 'AR': 38, 'AS': 1, 'AT': 6, 'RE': 2, 'AU': 131, 'AW': 1, 'AZ': 5, 'RO': 14, 'BA': 4, 'BB': 1, 'RS': 3, 'BD': 7, 'RU': 129, 'BE': 5, 'BF': 2, 'BG': 4, 'RW': 2, 'BH': 1, 'BI': 1, 'BJ': 1, 'BL': 1, 'BM': 1, 'BN': 1, 'BO': 15, 'SA': 26, 'SB': 17, 'BQ': 3, 'SC': 2, 'BR': 117, 'SD': 5, 'BS': 18, 'SE': 39, 'BT': 1, 'SG': 2, 'BW': 4, 'SH': 2, 'SI': 2, 'BY': 3, 'SK': 2, 'BZ': 13, 'SL': 1, 'SN': 4, 'SO': 6, 'CA': 205, 'SR': 1, 'SS': 1, 'CC': 1, 'CD': 11, 'ST': 1, 'CF': 1, 'SV': 1, 'CG': 3, 'CH': 6, 'SX': 1, 'CI': 2, 'SY': 2, 'SZ': 1, 'CK': 6, 'CL': 17, 'CM': 5, 'CN': 217, 'CO': 51, 'CR': 13, 'TC': 4, 'TD': 5, 'CU': 12, 'CV': 7, 'TG': 1, 'TH': 33, 'CW': 1, 'CX': 1, 'CY': 3, 'TJ': 4, 'CZ': 5, 'TL': 1, 'TM': 1, 'TN': 8, 'TO': 5, 'TR': 52, 'TT': 2, 'DE': 34, 'TV': 1, 'TW': 9, 'DJ': 1, 'TZ': 11, 'DK': 8, 'D